In [ ]:
!pip install tiktoken

In [ ]:
from importlib.metadata import version


print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1+cu121
tiktoken version: 0.8.0


In [ ]:
with open("/content/the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Total number of chatacters"  ,len(text))
print(text[:100])

Total number of chatacters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [ ]:
# this is  for the regx expressons

import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print(preprocessed[:100])
print(len(preprocessed))

# uniqe tokens in the data set
print(len(set(preprocessed)))


['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', '', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', ',', '', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting', ',', '', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow', ',', '', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ']
9235
1133


In [ ]:
# now we need to asign a unique id doe the indexing for each unique token

unique_tokens = list(sorted(set(preprocessed)))

vocab = {token:integer for integer,token in enumerate(unique_tokens)}


for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break


('', 0)
('\n', 1)
(' ', 2)
('!', 3)
('"', 4)
("'", 5)
('(', 6)
(')', 7)
(',', 8)
('--', 9)
('.', 10)
(':', 11)
(';', 12)
('?', 13)
('A', 14)
('Ah', 15)
('Among', 16)
('And', 17)
('Are', 18)
('Arrt', 19)
('As', 20)
('At', 21)
('Be', 22)
('Begin', 23)
('Burlington', 24)
('But', 25)
('By', 26)
('Carlo', 27)
('Chicago', 28)
('Claude', 29)
('Come', 30)
('Croft', 31)
('Destroyed', 32)
('Devonshire', 33)
('Don', 34)
('Dubarry', 35)
('Emperors', 36)
('Florence', 37)
('For', 38)
('Gallery', 39)
('Gideon', 40)
('Gisburn', 41)
('Gisburns', 42)
('Grafton', 43)
('Greek', 44)
('Grindle', 45)
('Grindles', 46)
('HAD', 47)
('Had', 48)
('Hang', 49)
('Has', 50)


In [ ]:
# we want a class to do this tockainzing and helper methodes to encode and decode
class SimpleTockainzerA:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {integer:token for token,integer in vocab.items()}

  def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

  def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
tockaizer = SimpleTockainzerA(vocab)



In [ ]:
tockaizer.encode("I HAD always thought Jack Gisburn rather a cheap?")

[56, 47, 152, 1006, 60, 41, 821, 118, 259, 13]

In [ ]:
tockaizer.decode([56, 47, 152, 1006, 60, 41, 821, 118, 259 ,13])

'I HAD always thought Jack Gisburn rather a cheap?'

In [8]:
# BytePair encoding
# Here now we are going to use the tiktoken tokenizer used as the
# tockenizer in GPT-2 this can also tockanize any new word also

!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.4 MB/s eta 0:00:00


In [9]:

# The importlib package is primarily utilized by Python applications for dynamic imports during runtime.
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [10]:
tokenizer = tiktoken.get_encoding("gpt2")

In [9]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

# here we can use the tokenizer for create the indexes
# here the  allowed_special={'<|endoftext|> part can be used to denote the end of the training
# of the dataset-1 and now we can start the dataset -2
integers = tokenizer.encode(text ,allowed_special={'<|endoftext|>'})
print(integers)


[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [10]:
# here we can use this now for the index decoding as well
decoded = tokenizer.decode(integers)
print(decoded)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [18]:
# BPE tokenizers break down unknown words into subwords and individual characters


encodede_data = tokenizer.encode("Akwirw ier" , allowed_special={'<|endoftext|>'})
for index , token_id in enumerate(encodede_data):
  token = tokenizer.decode([token_id])

  print(f"'{token}' , {token_id}")

'Ak' , 33901
'w' , 86
'ir' , 343
'w' , 86
' ' , 220
'ier' , 959


In [19]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [24]:
import torch.nn as nn
from supplementary import TransformerBlock, LayerNorm


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [25]:
model = GPTModel(GPT_CONFIG_124M)

In [1]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [32]:
# lets give a try
# step 1 - generrate the token indexes

start_text = "Hello this is a"
start_tokens = tokenizer.encode(start_text)

In [12]:
import torch

In [33]:
# convert te tokens in to a tensor
# Tensors are a specialized data structure that are very similar to arrays and matrices
endcoeded_tensors = torch.tensor(start_tokens).unsqueeze(0)

In [34]:
out = generate_text_simple(
    model=model,
    idx=endcoeded_tensors,
    max_new_tokens=6,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,   428,   318,   257, 16740, 41198, 49467, 46204,  1212, 22211]])
Output length: 10


In [35]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello this is arace Blend soluble worshippedThis echoed
